In [11]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

I_ = [(0.0 .. 1.1),(-0.5..0.8)]
c = mid.(I_)
f((x1,x2)) = x1^2-x2-1
g((x1,x2)) = (x1-2)^2 -x2-1
F((x,y))=[f((x,y));g((x,y))]
j = ForwardDiff.jacobian(F,I_)
R = inv(ForwardDiff.jacobian(F,c)) 
E = Array([1 0; 0 1])

K = c -R*(F(c))+(E - R*j)*(I_-c)

2-element Vector{Interval{Float64}}:
  [0.697499, 1.30251]
 [-0.807501, 0.402501]

In [49]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

b = [0.6593569186559406 0.1226894081314498 0.798454945194909; 0.18863873291688749 0.6437372095067467 0.49583295285327555; 0.33567285114178314 0.6471473112374553 0.2913368472779546]


3×3 Matrix{Float64}:
 0.659357  0.122689  0.798455
 0.188639  0.643737  0.495833
 0.335673  0.647147  0.291337

In [9]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff


function krawczyk(F,I_)
    c = mid.(I_)
    @show c
    j = ForwardDiff.jacobian(F,I_)
    @show j 
    R = inv(ForwardDiff.jacobian(F,c))
    @show R
    E = Matrix{Float64}(I,size(R))
    @show E
    K = c -R*(F(c))+(E - R*j)*(I_-c)
    return K
end

X = [(0.9..1.1),(-0.1..0.1)]

f((x1,x2)) = x1^2 - x2 - 1
g((x1,x2)) = (x1 - 2)^2 - x2 -1
F((x,y))=[f((x,y));g((x,y))]

println(krawczyk(F,X))



c = [1.0, 0.0]
j = Interval{Float64}[[1.79999, 2.20001] [-1, -1]; [-2.20001, -1.79999] [-1, -1]]
R = [0.25 -0.25; -0.5 -0.5]
E = [1.0 0.0; 0.0 1.0]
Interval{Float64}[[0.989999, 1.01001], [-0.0200001, 0.0200001]]


In [7]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = []
    for i = 1:size(X)[1]
        push!(L, intersect(X[i],Y[i]))
        # typeof(intersect(X[i],Y[i])) => Interval{Float64}
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=(0..0)
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end


function allsol(F,X)
    L = [X] #調査する区間のリスト
    S = [] #見つかった解のリスト
    cnt=0
    
    while (size(L)[1] != 0)
        X = popfirst!(L) #Lから区間を一つ取り出す
        @show X
        
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(c)+j*(X-c)) )
            continue
        end
    
        tmp = ForwardDiff.jacobian(F,c)
        
        R = tmp
        try
            R = inv(tmp)
        catch e
            println("逆行列の演算に失敗")
            println(e)
            return false
            # SingularExceptionが発生する
            # LAPACKExceptionが発生する
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        if (orig_is_empty(K,X))
            continue #解なし
        end
        
        if (orig_is_subset(K,X))　#解あり
            push!(S, K) #解のリストに追加
            continue
        end
        
        try
            X1,X2 = orig_devide(orig_intersect(K,X))　#K and X を2分割
            push!(L, X1)
            push!(L, X2)
        catch e
            println("区間の分割に失敗")
            println(e)
            return false
        end
        
    end
    
    return S
end

X = [(-1.1 .. 1.1),(-1.3 .. 1.1)]

# f((x1,x2))= x1^2 + x2^2 -1
# g((x1,x2)) = x1-x2

f((x1,x2)) = x1 + 2*x2 +1
g((x1,x2)) = x1^2 + 2*(x2^2) -1

F((x,y))=[f((x,y));g((x,y))]

li=allsol(F,X)

for i in li
    @show i
end



X = Interval{Float64}[[-1.10001, 1.10001], [-1.30001, 1.10001]]
X = Interval{Float64}[[-1.10001, 1.10001], [-1.30001, -0.1]]
X = Interval{Float64}[[-1.10001, 1.10001], [-0.1, 1.10001]]
X = Interval{Float64}[[-1.10001, 0], [-1.30001, -0.1]]
X = Interval{Float64}[[0, 1.10001], [-1.30001, -0.1]]
X = Interval{Float64}[[-1.10001, 0], [-0.1, 1.10001]]
X = Interval{Float64}[[0, 1.10001], [-0.1, 1.10001]]
X = Interval{Float64}[[-1.10001, 0], [-1.30001, -0.7]]
X = Interval{Float64}[[-1.10001, 0], [-0.700001, -0.1]]
X = Interval{Float64}[[0, 0.550001], [-1.08551, -0.267499]]
X = Interval{Float64}[[0.55, 1.10001], [-1.08551, -0.267499]]
X = Interval{Float64}[[-1.10001, -0.360714], [-0.1, 0.277084]]
X = Interval{Float64}[[-1.10001, -0.360714], [0.277083, 0.654167]]
X = Interval{Float64}[[-1.10001, -0.55], [-0.700001, -0.1]]
X = Interval{Float64}[[-0.550001, 0], [-0.700001, -0.1]]
X = Interval{Float64}[[0, 0.275001], [-0.88316, -0.452063]]
X = Interval{Float64}[[0.275, 0.550001], [-0.88316, -0.4520

In [13]:
    X=[(0..20),(0..30),(0..40)]   
    max_diam=(0..0)
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    @show X1,X2





(X1, X2) = (Interval{Float64}[[0, 20], [0, 30], [0, 20]], Interval{Float64}[[0, 20], [0, 30], [20, 40]])


(Interval{Float64}[[0, 20], [0, 30], [0, 20]], Interval{Float64}[[0, 20], [0, 30], [20, 40]])

In [6]:
function twosum(a::Float64,b::Float64)
    x=a+b
    if abs(a)>abs(b)
        tmp = x-a
        y = b-tmp
    else
        tmp = x-b
        y = a-tmp
    end
    return x , y
end

function split(a::Float64)
    tmp = a*((2. ^ 27)+1)
    x=tmp-(tmp-a)
    y=a-x
    return x,y
end

function twoproduct(a::Float64,b::Float64)
    x = a*b
    if abs(a)>2. ^ 996
        an=a*(2. ^ (-28))
        bn=b*(2. ^ 28)
    elseif abs(b)>2. ^ 996
        an=a*(2. ^ 28)
        bn=b*(2. ^ (-28))
    else
        an=a
        bn=b
    end 
    a1,a2 = split(a)
    b1,b2 = split(b)
    if abs(x)> 2. ^ 1023
        y = a2*b2-((((x*0.5)-(a1*0.5)+b1)*2-a2*b1)-a1*b2)
    else
        y = a2*b2-(((x-a1+b1)-a2*b1)-a1*b2)
    end
    
    return x,y
end



function succ(x::Float64)
    a = abs(x)
    if (a >= 2. ^ (-969))
        return x+a*(2. ^ (-53)+2. ^ (-105))
    end
    if (a < 2. ^ (-1021))
        return x+a*(2. ^ (-1074))    
    end
    c = (2. ^ 53)*x
    e = (2. ^ (-53))+(2. ^ (-105))*abs(c)
    return (c + e)*2. ^ (-53)
end
    
function pred(x::Float64)
    a = abs(x)
    if (a >= 2. ^ (-969))
        return x-a*(2. ^ (-53)+2. ^ (-105))
    end
    if (a < 2. ^ (-1021))
        return x-a*(2. ^ (-1074))    
    end
    c = (2. ^ 53)*x
    e = (2. ^ (-53))+(2. ^ (-105))*abs(c)
    return (c - e)*2. ^ (-53)
end
    
function add_up(a::Float64,b::Float64)
    x , y = twosum(a, b)
    if x == Inf
        return x
    elseif x == -Inf
        if a == -Inf || b == -Inf
            return x
        else
            return -prevfloat(typemax(Float64))
        end
    end
    if y > 0.
        x = succ(x)
    end
    return x
end

function add_down(a::Float64,b::Float64)
    x , y = twosum(a, b)
    if x == Inf
        if a == Inf || b == Inf
            return x
        else
            return prevfloat(typemax(Float64))
        end
    elseif x == -Inf
        return x
    end
    if y < 0.
        x = pred(x)
    end
    return x
end

function sub_up(a::Float64,b::Float64)
    x , y = twosum(a, -b)
    if x == Inf
        return x
    elseif x == -Inf
        if a == -Inf || b == -Inf
            return x
        else
            return -prevfloat(typemax(Float64))
        end
    end
    if y > 0.
        x = succ(x)
    end
    return x
end

function sub_down(a::Float64,b::Float64)
    x , y = twosum(a, -b)
    if x == Inf
        if a == Inf || b == Inf
            return x
        else
            return prevfloat(typemax(Float64))
        end
    elseif x == -Inf
        return x
    end
    if y < 0.
        x = pred(x)
    end
    return x
end

function mul_up(a::Float64, b::Float64)
    x , y = twoproduct(a, b)
    if x == Inf
        return x
    elseif x == -Inf
        if abs(a) == Inf || abs(b) == Inf
            return x
        else
            return -prevfloat(typemax(Float64))
        end
    end
    
    if abs(x)>2. ^ (-969)
        if y > 0
            return succ(x)
        end
        return x
    else
        s,s2 = twoproduct(a * 2. ^ 537, b * 2. ^ 537)
        t = (x * 2. ^ 537) * 2. ^537
        if t<s || (t==s && s2>0)
            return succ(x)
        end
        return x
    end
end

function mul_down(a::Float64, b::Float64)
    x , y = twoproduct(a, b)
    if x == Inf
        if abs(a) == Inf || abs(b) == Inf
            return x
        else
            return prevfloat(typemax(Float64))
        end
    elseif x == -Inf
        return x
    end
    
    if abs(x)>2. ^ (-969)
        if y<0
            return pred(x)
        end
        return x
    else
        s,s2 = twoproduct(a * 2. ^ 537, b * 2. ^ 537)
        t = (x * 2. ^ 537) * 2. ^537
        if t>s || (t==s && s2<0)
            return pred(x)
        end
        return x
    end
end

function div_up(a::Float64, b::Float64)
    if a==0 || b==0 || abs(a) == Inf || abs(b) == Inf || a!=a || b!=b
        return a/b
    end
    
    if b<0
        an,bn = -a,-b
    else
        an,bn = a,b
    end
    
    if abs(an) < 2. ^ (-969)
        if abs(bn) < 2. ^ 918
            an *= 2. ^ 105
            bn *= 2. ^ 105
        else
            if an<0
                return 0
            else
                return 2. ^ (-1074)
            end
        end
    end
    
    d= an/bn
    
    if d == Inf
        return d
    elseif d == -Inf
        return -prevfloat(typemax(Float64))
    end
    
    x,y = twoproduct(d,bn)
    if x < an || (x == an && y < 0)
        return succ(d)
    end
    return d
end

function div_down(a::Float64, b::Float64)
    if a==0 || b==0 || abs(a) == Inf || abs(b) == Inf || a!=a || b!=b
        return a/b
    end
    
    if b<0
        an,bn = -a,-b
    else
        an,bn = a,b
    end
    
    if abs(an) < 2. ^ (-969)
        if abs(bn) < 2. ^ 918
            an *= 2. ^ 105
            bn *= 2. ^ 105
        else
            if an<0
                return -2. ^ (-1074)
            else
                return 0.
            end
        end
    end
    
    d= an/bn
    
    if d == Inf
        return prevfloat(typemax(Float64))
    elseif d == -Inf
        return d
    end
    
    x,y = twoproduct(d,bn)
    if x > an || (x == an && y > 0)
        return pred(d)
    end
    return d
end

function sqrt_up(a::Float64)
    d = sqrt(a)
    if a < 2. ^ (-969)
        a2 = a*(2. ^ 106)
        d2 = d*(2. ^ 53)
        x,y = twoproduct(d2,d2)
        if x < a2 || (x == a2 && y < 0)
            d = succ(d)
        end
        return d
    end
    
    x,y = twoproduct(d,d)
    if x < a || (x == a && y < 0.)
        d = succ(d)
    end
    return d
end

function sqrt_down(a::Float64)
    d = sqrt(a)
    if a < 2. ^ (-969)
        a2 = a*(2. ^ 106)
        d2 = d*(2. ^ 53)
        x,y = twoproduct(d2,d2)
        if x > a2 || (x == a2 && y > 0)
            d = pred(d)
        end
        return d
    end
    
    x,y = twoproduct(d,d)
    if x > a || (x == a && y > 0.)
        d = pred(d)
    end
    return dprintln(mul_up(0.1,3.))
end

sqrt_down (generic function with 1 method)